<a href="https://colab.research.google.com/github/ericodle/mrbayes_primer/blob/main/MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import random
import numpy as np

In [26]:
fasta_content = '''>Sequence1
ACGTACGTACGTACGT
>Sequence2
TGCATGCATGCATGCA
>Sequence3
ATCGATCGATCGATCG
>Sequence4
TAGCTAGCTAGCTAGC
>Sequence5
GCTAGCTAGCTAGCTA'''

file_name = 'sequences.fasta'

with open(file_name, 'w') as file:
    file.write(fasta_content)

print(f"FASTA file '{file_name}' has been created.")


FASTA file 'sequences.fasta' has been created.


In [27]:
# Load sequences from a FASTA file
def load_sequences(fasta_file):
    sequences = []
    with open(fasta_file, 'r') as file:
        for line in file:
            if line.startswith('>'):
                continue
            sequence = line.strip()
            sequences.append(sequence)
    return sequences

In [28]:
# Compute the Jukes-Cantor distance between two sequences
def compute_distance(sequence1, sequence2):
    mismatches = sum(base1 != base2 for base1, base2 in zip(sequence1, sequence2))
    denominator = (4 / 3) * (mismatches / len(sequence1))
    if denominator == 1:
        return float('inf')
    elif denominator < 1:
        return -0.75 * np.log(1 - denominator)
    else:
        return np.inf

This function calculates the Jukes-Cantor distance between two DNA sequences. It counts the number of mismatches between the sequences and uses the Jukes-Cantor formula to compute the distance. If the denominator of the formula is zero or negative, it handles those cases appropriately. The function returns the Jukes-Cantor distance.

In [29]:
# Example function for updating substitution counts
def update_substitution_counts(substitution_rates, substitution_counts):
    for i in range(4):
        substitution_counts[i] += substitution_rates[i]

This function is an example implementation to update the count matrices for substitution rates. It takes the calculated substitution rates and updates the corresponding count matrix by adding the rates.

In [30]:
# Example function for computing substitution rates
def compute_substitution_rates(distance_matrix):
    # Compute average distance for each pair of bases
    average_distance = np.mean(distance_matrix, axis=0)

    # Compute substitution rates using Jukes-Cantor model
    substitution_rates = np.exp(-average_distance)

    return substitution_rates

This function is an example implementation to compute substitution rates from the distance matrix. In the Jukes-Cantor model, the substitution rates are derived by exponentiating the average distance values. However, since the code has been updated to use the GTR model, this function is no longer used.

In [31]:
# Perform MCMC sampling with burn-in and calculate posterior probabilities
def run_mcmc_sampling(sequences, num_iterations, burn_in):
    n = len(sequences)
    sequence_length = len(sequences[0])

    # Initialize the distance matrix
    distance_matrix = np.zeros((n, n))

    # Initialize count matrices for substitution rates
    substitution_counts = np.zeros((4, 4))

    # Run MCMC iterations with burn-in
    for iteration in range(num_iterations):
        # Randomly choose two sequences
        seq_idx1, seq_idx2 = random.sample(range(n), 2)

        # Generate new sequence based on existing sequences
        new_seq = ''.join(random.choice(sequences[seq_idx1] + sequences[seq_idx2]) for _ in range(sequence_length))

        # Compute distance of the new sequence with other sequences
        for i in range(n):
            if i == seq_idx1 or i == seq_idx2:
                continue
            new_distance = compute_distance(new_seq, sequences[i])
            old_distance = distance_matrix[seq_idx1, i] + distance_matrix[seq_idx2, i]
            distance_matrix[seq_idx1, i] = distance_matrix[i, seq_idx1] = new_distance
            distance_matrix[seq_idx2, i] = distance_matrix[i, seq_idx2] = new_distance
            distance_matrix[seq_idx1, seq_idx2] = distance_matrix[seq_idx2, seq_idx1] = old_distance

        # Apply burn-in by discarding distances during burn-in period
        if iteration >= burn_in:
            # Compute substitution rates from distance matrix
            substitution_rates = compute_substitution_rates(distance_matrix)

            # Update substitution counts
            update_substitution_counts(substitution_rates, substitution_counts)

    # Calculate posterior probabilities based on substitution counts
    total_counts = np.sum(substitution_counts)
    substitution_probs = substitution_counts / total_counts

    # Calculate prior probabilities based on uniform distribution
    prior_probs = np.full_like(substitution_probs, 1 / 16)

    # Calculate denominator for Bayes' formula
    denominator = np.sum(substitution_probs * prior_probs)

    # Calculate posterior probabilities using Bayes' formula
    posterior_probs = (substitution_probs * prior_probs) / denominator

    # Print the posterior probabilities
    print("Posterior probabilities:")
    print(posterior_probs)

This is the main function that performs MCMC sampling to estimate substitution rates and posterior probabilities. It takes the DNA sequences, the number of iterations to run, and the burn-in period as input. Within the function, it initializes the distance matrix and count matrices for substitution rates. It then iterates over the specified number of iterations, randomly selects two sequences, generates a new sequence, computes distances, and updates the distance matrix and substitution count matrices. After the burn-in period, it calculates substitution rates, posterior probabilities, and prints the posterior probabilities.

In [32]:
# Main code
fasta_file = "sequences.fasta"
sequences = load_sequences(fasta_file)

num_iterations = 1000
burn_in = 100

run_mcmc_sampling(sequences, num_iterations, burn_in)

Posterior probabilities:
[[0.06656232 0.06656232 0.06656232 0.06656232]
 [0.09443965 0.09443965 0.09443965 0.09443965]
 [0.03835605 0.03835605 0.03835605 0.03835605]
 [0.05064198 0.05064198 0.05064198 0.05064198]]


The posterior probability matrix represents the probabilities of different nucleotide substitutions occurring at each position in the DNA sequences. Each element in the matrix corresponds to a specific substitution from one nucleotide to another at a specific position.

While the code example provides a simplified demonstration of MCMC sampling and the computation of posterior probabilities, it does not provide the full range of output and analysis features offered by a comprehensive software package like MrBayes. MrBayes is a sophisticated software tool specifically designed for Bayesian phylogenetic analysis, offering a wide range of features and extensive output for interpreting and analyzing the results.